## This notebook contains the classification procedure for the reduced FFT approach

In [54]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
import torch
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout,Activation,MaxPooling2D,ReLU,Add,GlobalAvgPool2D
from tensorflow.keras import Model,Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import ast

## Get the Data and Labels

In [55]:
df_norm = pd.read_csv('data/data_prepared/data_norm_reduced/data_reduced_norm.csv')

In [56]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [57]:
valence.shape

(1280,)

In [58]:
df_norm

,p:sub01 v:0,p:sub01 v:1,p:sub01 v:2,p:sub01 v:3,p:sub01 v:4,p:sub01 v:5,p:sub01 v:6,p:sub01 v:7,p:sub01 v:8,p:sub01 v:9,...,p:sub32 v:30,p:sub32 v:31,p:sub32 v:32,p:sub32 v:33,p:sub32 v:34,p:sub32 v:35,p:sub32 v:36,p:sub32 v:37,p:sub32 v:38,p:sub32 v:39
0,"[0.4485464836863576, 0.23743089577886559, 0.25...","[0.7019180566921461, 0.5461401369897261, 0.551...","[0.6288997216273957, 0.5353532230667453, 0.539...","[0.6266704748094754, 0.47343463832216404, 0.22...","[0.0, 0.027719349129351294, 0.2419823433081025...","[0.4664035232446203, 0.27838186329410847, 0.30...","[0.6703302868325047, 0.32789231857853357, 0.66...","[0.9899844902094569, 0.810246175123128, 0.8003...","[1.0, 0.8200701167617226, 0.7860376730186085, ...","[0.4504604599247114, 0.26668218946707795, 0.54...",...,"[0.023487912614575716, 0.0, 0.1688471178401590...","[0.5422144993423434, 0.5920108093137332, 0.739...","[0.7929219302890472, 0.5110720072159004, 0.446...","[0.2989311031697405, 0.39466039348987386, 0.39...","[0.383499595408718, 0.1780261984830269, 0.2179...","[0.42486739015137676, 0.38144560948409556, 0.6...","[0.8302330838047564, 0.6761027213858244, 0.686...","[0.4694137889505188, 0.3410767456862581, 0.857...","[0.3528522856935695, 0.13774190624327134, 0.37...","[0.14069406842073867, 0.06540498312891915, 0.0..."
1,"[0.3881421834000899, 0.25757925871200654, 0.12...","[0.6446117327907707, 0.4290718164937975, 0.418...","[0.6567605589406893, 0.4760953784058103, 0.630...","[0.6893590796863427, 0.5821362678720333, 0.167...","[0.0, 0.0, 0.013691529830546565, 0.0, 0.020032...","[0.38138185486796766, 0.21776423581608575, 0.1...","[0.6475523502534886, 0.36071087386915485, 0.68...","[0.7217503873066675, 0.6183223855997101, 0.715...","[1.0, 0.7796422878285081, 0.9165695334272908, ...","[0.43625249302074903, 0.25614302060794736, 0.4...",...,"[0.13805938263760228, 0.14482431311589208, 0.0...","[0.42743101453775434, 0.3145156237450833, 0.57...","[0.8174801289722773, 0.6648842525767291, 0.454...","[0.4893012398936179, 0.4615107081002191, 0.249...","[0.456932248157847, 0.2525345242361623, 0.0412...","[0.6239491539592579, 0.5348941954638722, 0.424...","[0.6401933134530025, 0.5856376132877901, 0.750...","[0.4079077498823079, 0.26794531113606024, 0.71...","[0.17993968035747648, 0.10303093005529375, 0.2...","[0.07519943026129747, 0.06811714955871212, 0.0..."
2,"[0.3740623804802889, 0.3928083563481559, 0.160...","[0.6973291997292141, 0.5943537959674001, 0.449...","[0.7436668480597128, 0.660162877660154, 0.5590...","[0.6379264861826297, 0.6374250421627957, 0.258...","[0.0, 0.0, 0.031795570390553814, 0.0, 0.001354...","[0.4200321461199312, 0.33768893694259255, 0.17...","[0.6757609382444814, 0.6536756912080274, 0.405...","[0.8007048896273071, 0.8564056310039908, 0.945...","[1.0, 1.0, 0.907485307279899, 0.78238725106086...","[0.4311913649293486, 0.39919134912246734, 0.29...",...,"[0.2232974764757198, 0.21051172045885758, 0.05...","[0.44460813163064694, 0.5084187893224926, 0.51...","[0.6096025980861909, 0.6317427119375005, 0.474...","[0.46830142181493184, 0.4117710565765322, 0.43...","[0.5029684246676773, 0.3616708535292109, 0.191...","[0.4514800555139707, 0.4947761885881989, 0.722...","[0.7982065081344615, 0.8092938704287668, 1.0, ...","[0.3109242170552014, 0.37955803385809894, 0.25...","[0.2946999836785654, 0.23458595195513507, 0.18...","[0.08253357948534391, 0.04238300103831488, 0.0..."
3,"[0.43305115456159876, 0.33820919608068983, 0.2...","[0.7272727360986004, 0.4013172120922402, 0.324...","[0.628193005579958, 0.42193623282761633, 0.360...","[0.7348494713934763, 0.45388918362867675, 0.13...","[0.0, 0.0, 0.08854355459381283, 0.242177251600...","[0.4408158590722382, 0.1922719503295613, 0.166...","[0.600681616967235, 0.36074166602101265, 0.504...","[0.8029607741233296, 0.5901930115923396, 0.602...","[1.0, 0.7598807859499533, 0.546100683704346, 0...","[0.42463990910031324, 0.17976566651074732, 0.2...",...,"[0.22602732143513016, 0.0926236454061902, 0.0,...","[0.4947834778732565, 0.31396

In [61]:
data = []
for i in range(880):
    list_array = np.array([ast.literal_eval(string) for string in df_norm.iloc[:,i]])
    values = np.concatenate(list_array, axis=0).reshape(16, 8)
    data.append(values)

In [62]:
data[0].shape

(16, 8)

In [63]:
data[0]

array([[0.44854648, 0.2374309 , 0.25671829, 0.13548509, 0.18036478,
        0.41318234, 0.08743186, 0.08291412],
       [0.38814218, 0.25757926, 0.12327163, 0.0932041 , 0.27478092,
        0.37156167, 0.14416161, 0.11629708],
       [0.37406238, 0.39280836, 0.16097329, 0.28599972, 0.10682097,
        0.24319822, 0.03861371, 0.02806835],
       [0.43305115, 0.3382092 , 0.2658135 , 0.34060688, 0.0820547 ,
        0.22157597, 0.03515119, 0.02825133],
       [0.33663758, 0.18796175, 0.199303  , 0.2086844 , 0.27466817,
        0.32792286, 0.13392667, 0.17481906],
       [0.41226243, 0.46094699, 0.33397622, 0.22673163, 0.19849791,
        0.39888367, 0.08293107, 0.08334848],
       [0.32646167, 0.23408946, 0.16471522, 0.1750405 , 0.19852092,
        0.0920969 , 0.18221539, 0.16349722],
       [0.33796216, 0.41936573, 0.48085898, 0.20918281, 0.09555173,
        0.36320586, 0.0525679 , 0.04218279],
       [0.34612107, 0.23118263, 0.28974674, 0.20263353, 0.22016241,
        0.4574386 , 0.103502

## Create train test split

### Data

In [64]:
df_norm.shape[1]

1280

In [65]:
# training with 22 participants each has 40 videos = 880
train = np.zeros((0,16,8))
train[:] = np.nan
#train1 = 
test = np.zeros((0,16,8))
test[:] = np.nan

for i in range(df_norm.shape[1]):
    if i < 880:
        list_array = np.array([ast.literal_eval(string) for string in df_norm.iloc[:,i]])
        values = np.concatenate(list_array, axis=0).reshape(16, 8)
        values = values.reshape(1,16,8)
        train = np.concatenate((train, values), axis=0)
    else:
        list_array = np.array([ast.literal_eval(string) for string in df_norm.iloc[:,i]])
        values = np.concatenate(list_array, axis=0).reshape(16, 8)
        values = values.reshape(1,16,8)
        test = np.concatenate((test, values), axis=0)
    

In [66]:
np.isnan(train).any()

False

In [67]:
np.isnan(test).any()

False

In [68]:
train.shape

(880, 16, 8)

In [69]:
test.shape

(400, 16, 8)

### Get generated Data

### 01 - x2

In [70]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled/red/02/a/sampled_fftreduced_100000_128_"+str(i),map_location=torch.device('cpu')))
generated_data = np.concatenate(arrays, axis=0)

In [71]:
generated_data.shape

(1280, 1, 16, 8)

In [72]:
X_argument2 = np.zeros((880,16,8))
X_argument2[:] = np.nan

for i in range(880):
    X_argument2[i] = generated_data[i][0][:,:] 

In [73]:
X_argument2.shape

(880, 16, 8)

In [74]:
# 2x
x_2 = np.concatenate([X_argument2, train], 0) 
print(x_2.shape)

(1760, 16, 8)


### 01 - x3

In [75]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled/red/02/b/sampled_fftreduced_100000_128_"+str(i),map_location=torch.device('cpu')))
generated_data = np.concatenate(arrays, axis=0)

In [76]:
generated_data.shape

(1280, 1, 16, 8)

In [77]:
X_argument3 = np.zeros((880,16,8))
X_argument3[:] = np.nan

for i in range(880):
    X_argument3[i] = generated_data[i][0][:,:] 

In [78]:
X_argument3.shape

(880, 16, 8)

In [79]:
# 3x
x_3 = np.concatenate([X_argument2, X_argument3, train], 0) 
print(x_3.shape)

(2640, 16, 8)


### 01 - x4

In [80]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled/red/02/c/sampled_fftreduced_100000_128_"+str(i),map_location=torch.device('cpu')))
generated_data = np.concatenate(arrays, axis=0)

In [81]:
generated_data.shape

(1280, 1, 16, 8)

In [82]:
X_argument4 = np.zeros((880,16,8))
X_argument4[:] = np.nan

for i in range(880):
    X_argument4[i] = generated_data[i][0][:,:] 

In [83]:
X_argument2.shape

(880, 16, 8)

In [84]:
# 4x
x_4 = np.concatenate([X_argument2, X_argument3, X_argument4, train], 0) 
print(x_4.shape)

(3520, 16, 8)


### Labels

In [85]:
val22 = valence[: 880]
aro22 = arousal[: 880]
dom22 = dominance[: 880]
lik22 = liking[: 880]

y_test_va = valence[880: ]
y_test_ar = arousal[880: ]
y_test_do = dominance[880: ]
y_test_li = liking[880: ]

In [86]:
# Augmenting the labels 
# 2x
yva2 = np.hstack((val22,val22))
yar2 = np.hstack((aro22,aro22))
ydo2 = np.hstack((dom22,dom22))
yli2 = np.hstack((lik22,lik22))


# 3x
yva3 = np.hstack((val22,val22,val22))
yar3 = np.hstack((aro22,aro22,aro22))
ydo3 = np.hstack((dom22,dom22,dom22))
yli3 = np.hstack((lik22,lik22,lik22))

# 4x
yva4 = np.hstack((val22,val22,val22,val22))
yar4 = np.hstack((aro22,aro22,aro22,aro22))
ydo4 = np.hstack((dom22,dom22,dom22,dom22))
yli4 = np.hstack((lik22,lik22,lik22,lik22))

In [87]:
print(val22.shape)
print(aro22.shape)
print(dom22.shape)
print(lik22.shape)
print(y_test_va.shape)
print(y_test_ar.shape)
print(y_test_do.shape)
print(y_test_li.shape)

(880,)
(880,)
(880,)
(880,)
(400,)
(400,)
(400,)
(400,)


## KNN

In [88]:
train.reshape(-1, 16*8).shape

(880, 128)

In [89]:
# Define KNN based model for actual/original dataset
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  random_state=100
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtrain.shape,ytrain.shape)
  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval) 
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [90]:
# Define KNN based model for augmented data
def knnmodel2(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  #print(classification_report(ytest, y_pred))
  #print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [91]:
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
score = []


for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(train.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,kval = k)
    scorelist.append((sc,k))
    score.append(sc)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

#for i,name in zip(scorelist,names):
#    print ("Accuracy of ",name ,i)

K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 74.0
K value = 20
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 63.0
######################################### Liking #########################################
Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 66.5
######################################### Liking #########################################
Accuracy 74.0
K value = 22
######################################### Valence #########################################
Acc

Accuracy 65.0
######################################### Liking #########################################
Accuracy 69.0
K value = 40
######################################### Valence #########################################
Accuracy 71.75
######################################### Arousal #########################################
Accuracy 54.0
######################################### Dominance #########################################
Accuracy 64.0
######################################### Liking #########################################
Accuracy 69.0
K value = 41
######################################### Valence #########################################
Accuracy 71.75
######################################### Arousal #########################################
Accuracy 54.0
######################################### Dominance #########################################
Accuracy 65.0
######################################### Liking #########################################
Accuracy 69.0
K v

In [93]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(77.75, 25)
(65.0, 14)
(69.5, 13)
(75.0, 16)


### 2X Data

In [94]:
print("Augmented data x2 shape:",x_2.shape)
print(yva2.shape,yar2.shape,ydo2.shape,yli2.shape)

Augmented data x2 shape: (1760, 16, 8)
(1760,) (1760,) (1760,) (1760,)


In [95]:
trainlabel_list = [yva2, yar2, ydo2, yli2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    # before x22
    sc = knnmodel2(x_2.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 76.75
######################################### Arousal #########################################
Accuracy 63.5
######################################### Dominance #########################################
Accuracy 63.5
######################################### Liking #########################################
Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 65.5
######################################### Liking #########################################
Accuracy 75.0
K value = 22
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
#

Accuracy 71.75
######################################### Arousal #########################################
Accuracy 55.50000000000001
######################################### Dominance #########################################
Accuracy 65.5
######################################### Liking #########################################
Accuracy 72.5
K value = 40
######################################### Valence #########################################
Accuracy 71.75
######################################### Arousal #########################################
Accuracy 54.50000000000001
######################################### Dominance #########################################
Accuracy 63.5
######################################### Liking #########################################
Accuracy 72.5
K value = 41
######################################### Valence #########################################
Accuracy 70.25
######################################### Arousal ###############################

In [97]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(79.75, 7)
(65.0, 14)
(67.5, 47)
(75.0, 10)


### 3X Data

In [98]:
trainlabel_list = [yva3, yar3, ydo3, yli3]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(x_3.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.0
######################################### Dominance #########################################
Accuracy 64.5
######################################### Liking #########################################
Accuracy 75.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.0
######################################### Dominance #########################################
Accuracy 66.5
######################################### Liking #########################################
Accuracy 75.0
K value = 22
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
#

Accuracy 65.0
######################################### Liking #########################################
Accuracy 70.5
K value = 40
######################################### Valence #########################################
Accuracy 71.25
######################################### Arousal #########################################
Accuracy 54.50000000000001
######################################### Dominance #########################################
Accuracy 63.5
######################################### Liking #########################################
Accuracy 70.5
K value = 41
######################################### Valence #########################################
Accuracy 71.75
######################################### Arousal #########################################
Accuracy 54.50000000000001
######################################### Dominance #########################################
Accuracy 66.0
######################################### Liking #################################

In [99]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(78.25, 27)
(64.5, 17)
(67.5, 49)
(75.0, 5)


### 4X Data

In [55]:
x_4.shape

(3520, 40, 99)

In [41]:
trainlabel_list = [yva4, yar4, ydo4, yli4]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(x_4.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 57.75
######################################### Arousal #########################################
Accuracy 52.5
######################################### Dominance #########################################
Accuracy 61.5
######################################### Liking #########################################
Accuracy 63.0
K value = 2
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 49.0
######################################### Dominance #########################################
Accuracy 53.0
######################################### Liking #########################################
Accuracy 70.5
K value = 3
######################################### Valence #########################################
Accuracy 71.75
#####

Accuracy 61.0
######################################### Liking #########################################
Accuracy 74.0
K value = 20
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 60.5
######################################### Liking #########################################
Accuracy 74.0
K value = 21
######################################### Valence #########################################
Accuracy 76.75
######################################### Arousal #########################################
Accuracy 64.5
######################################### Dominance #########################################
Accuracy 62.5
######################################### Liking #########################################
Accuracy 74.0
K v

Accuracy 65.5
######################################### Liking #########################################
Accuracy 70.5
K value = 39
######################################### Valence #########################################
Accuracy 71.75
######################################### Arousal #########################################
Accuracy 55.00000000000001
######################################### Dominance #########################################
Accuracy 65.5
######################################### Liking #########################################
Accuracy 70.5
K value = 40
######################################### Valence #########################################
Accuracy 71.75
######################################### Arousal #########################################
Accuracy 54.50000000000001
######################################### Dominance #########################################
Accuracy 62.0
######################################### Liking #################################

In [42]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(77.25, 14)
(64.5, 18)
(67.5, 33)
(75.0, 22)


## SVM

In [100]:
# Define SVM based model for augmented data
def svmmodel(xtrain,ytrain,xtest,ytest,c,kernel = 'linear'):

  # SVM model

  model = SVC(C=c,kernel=kernel,gamma = 'auto')
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [101]:
# Using actual data
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(train.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

### 2X Data

In [102]:
# Using 2X data
trainlabel_list = [yva2, yar2, ydo2, yli2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x_2.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

### 3X Data

In [110]:
# Using 3X data
trainlabel_list = [yva3, yar3, ydo3, yli3]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x_3.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

### 4X Data

In [104]:
# Using 4X data
trainlabel_list = [yva4, yar4, ydo4, yli4]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x_4.reshape(-1, 16*8),traain,test.reshape(-1, 16*8),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
Ac

### Neural Network

In [105]:
# Split the data into training/testing sets
def deepmodel(modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size):
  # ()
  
  ytrain = to_categorical(ytrain)
  ytest = to_categorical(ytest)



  input_shape = (16, 8, 1)
  # print(xtrain.shape,xtest.shape)
  xtrain = np.reshape(xtrain,[-1, 16, 8, 1])
  xtest = np.reshape(xtest,[-1, 16, 8, 1])
  print("xtrain.shape,ytrain.shape")
  print(xtrain.shape,ytrain.shape)
  if modelnum == 1:
  # Model 1
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # model.add(Dense(1, activation='sigmoid'))
  elif modelnum == 2:
    # Resnet kind of model

    def conv_bn_relu(x, filters, kernel_size, strides):
        x = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x


    def identity_block(tensor, filters):
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=1)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        x = Add()([x, tensor])
        x = ReLU()(x)
        return x


    def projection_block(tensor, filters, strides):
        # left branch
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=strides)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        # right branch
        shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x


    def resnet_block(x, filters, reps, strides):
        x = projection_block(x, filters=filters, strides=strides)
        for _ in range(reps-1):  # the -1 is because the first block was a Conv one
            x = identity_block(x, filters=filters)
        return x


    input = Input(shape=(32, 8, 1))

    x = conv_bn_relu(input, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2

    x = resnet_block(x, filters=64, reps=3, strides=1)
    x = resnet_block(x, filters=128, reps=4, strides=2)  # s=2 ([2]: conv3_1)
    x = resnet_block(x, filters=256, reps=6, strides=2)  # s=2 ([2]: conv4_1)
    x = resnet_block(x, filters=512, reps=3, strides=2)  # s=2 ([2]: conv5_1)

    x = GlobalAvgPool2D()(x)  # [3]: average pool *it is not written any pool size so we use Global

    # # output = Dense(1, activation='sigmoid')(x)
    output = Dense(2, activation='softmax')(x)
    model = Model(input, output)



  model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy','categorical_crossentropy'
                    optimizer=Adam(),
                    metrics=['accuracy'])
  # Fit data to model
  history = model.fit(xtrain,ytrain ,
            batch_size = batch_size,
            epochs=nepochs,
            verbose=1)

  # Generate generalization metrics
  scores = model.evaluate(xtest, ytest, verbose=0)#verbose =1 
  y_pred = model.predict(xtest)
  #     print("Fold number",fold_no)
  acc = scores[1]*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print("Loss",scores[0])
  # print(classification_report(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  # print(confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  return acc

## Normal Data

In [106]:
# Using original data
trainlabel_list = [val22, aro22, dom22, lik22]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,train.reshape(-1, 16,8),traain,test.reshape(-1, 16,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(880, 16, 8, 1) (880, 2)
Epoch 1/200
28/28 [==============================] - 2s 5ms/step - loss: 0.6816 - accuracy: 0.5545
Epoch 2/200
28/28 [==============================] - 0s 5ms/step - loss: 0.6125 - accuracy: 0.6807
Epoch 3/200
28/28 [==============================] - 0s 5ms/step - loss: 0.5439 - accuracy: 0.7466
Epoch 4/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4976 - accuracy: 0.7852
Epoch 5/200
28/28 [==============================] - 0s 5ms/step - loss: 0.5036 - accuracy: 0.7909
Epoch 6/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4855 - accuracy: 0.7852
Epoch 7/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4895 - accuracy: 0.7693
Epoch 8/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4807 - accuracy: 0.7818
Epoch 9/200
28/28 [==============================] - 0s 4ms/step 

28/28 [==============================] - 0s 5ms/step - loss: 0.4608 - accuracy: 0.7966
Epoch 163/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4620 - accuracy: 0.7966
Epoch 164/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4612 - accuracy: 0.7966
Epoch 165/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4602 - accuracy: 0.7966
Epoch 166/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4605 - accuracy: 0.7966
Epoch 167/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4604 - accuracy: 0.7966
Epoch 168/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4604 - accuracy: 0.7966
Epoch 169/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4604 - accuracy: 0.7966
Epoch 170/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4612 - accuracy: 0.7966
Epoch 171/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4602 - accuracy: 0.7966
Epoc

28/28 [==============================] - 0s 4ms/step - loss: 0.5949 - accuracy: 0.6955
Epoch 123/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5969 - accuracy: 0.6909
Epoch 124/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5933 - accuracy: 0.6932
Epoch 125/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5933 - accuracy: 0.6943
Epoch 126/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5966 - accuracy: 0.6886
Epoch 127/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5949 - accuracy: 0.6943
Epoch 128/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5948 - accuracy: 0.6943
Epoch 129/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5949 - accuracy: 0.6932
Epoch 130/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5948 - accuracy: 0.6943
Epoch 131/200
28/28 [==============================] - 0s 4ms/step - loss: 0.5961 - accuracy: 0.6852
Epoc

28/28 [==============================] - 1s 4ms/step - loss: 0.6935 - accuracy: 0.5466
Epoch 2/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6840 - accuracy: 0.5580
Epoch 3/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6806 - accuracy: 0.5466
Epoch 4/200
28/28 [==============================] - 0s 5ms/step - loss: 0.6607 - accuracy: 0.6148
Epoch 5/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6613 - accuracy: 0.6068
Epoch 6/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6485 - accuracy: 0.6170
Epoch 7/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6379 - accuracy: 0.6477
Epoch 8/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6341 - accuracy: 0.6398
Epoch 9/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6226 - accuracy: 0.6568
Epoch 10/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6225 - accuracy: 0.6443
Epoch 11/200
28/28 [=

28/28 [==============================] - 0s 4ms/step - loss: 0.6050 - accuracy: 0.6636
Epoch 165/200
28/28 [==============================] - 0s 5ms/step - loss: 0.6051 - accuracy: 0.6693
Epoch 166/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6031 - accuracy: 0.6670
Epoch 167/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6051 - accuracy: 0.6750
Epoch 168/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6047 - accuracy: 0.6716
Epoch 169/200
28/28 [==============================] - 0s 5ms/step - loss: 0.6034 - accuracy: 0.6648
Epoch 170/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6051 - accuracy: 0.6727
Epoch 171/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6029 - accuracy: 0.6761
Epoch 172/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6042 - accuracy: 0.6727
Epoch 173/200
28/28 [==============================] - 0s 4ms/step - loss: 0.6047 - accuracy: 0.6773
Epoc

28/28 [==============================] - 0s 5ms/step - loss: 0.4919 - accuracy: 0.7682
Epoch 125/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4921 - accuracy: 0.7705
Epoch 126/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4937 - accuracy: 0.7625
Epoch 127/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4936 - accuracy: 0.7773
Epoch 128/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4914 - accuracy: 0.7705
Epoch 129/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4914 - accuracy: 0.7739
Epoch 130/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4923 - accuracy: 0.7739
Epoch 131/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4923 - accuracy: 0.7727
Epoch 132/200
28/28 [==============================] - 0s 4ms/step - loss: 0.4902 - accuracy: 0.7739
Epoch 133/200
28/28 [==============================] - 0s 5ms/step - loss: 0.4907 - accuracy: 0.7773
Epoc

### 2x Data

In [107]:
# Using 2x data
trainlabel_list = [yva2, yar2, ydo2, yli2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x_2.reshape(-1, 16,8),traain,test.reshape(-1, 16,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(1760, 16, 8, 1) (1760, 2)
Epoch 1/200
55/55 [==============================] - 2s 6ms/step - loss: 0.6883 - accuracy: 0.5403
Epoch 2/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6794 - accuracy: 0.5699
Epoch 3/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6339 - accuracy: 0.6307
Epoch 4/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6072 - accuracy: 0.6415
Epoch 5/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6036 - accuracy: 0.6477
Epoch 6/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5951 - accuracy: 0.6500
Epoch 7/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5882 - accuracy: 0.6517
Epoch 8/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5897 - accuracy: 0.6562
Epoch 9/200
55/55 [==============================] - 0s 5ms/ste

55/55 [==============================] - 0s 6ms/step - loss: 0.5573 - accuracy: 0.7017
Epoch 163/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5598 - accuracy: 0.6966
Epoch 164/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5592 - accuracy: 0.6994
Epoch 165/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5578 - accuracy: 0.6972
Epoch 166/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5566 - accuracy: 0.7000
Epoch 167/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5590 - accuracy: 0.6983
Epoch 168/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5595 - accuracy: 0.6994
Epoch 169/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5582 - accuracy: 0.6983
Epoch 170/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5570 - accuracy: 0.6983
Epoch 171/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5584 - accuracy: 0.7000
Epoc

55/55 [==============================] - 0s 5ms/step - loss: 0.6206 - accuracy: 0.6557
Epoch 123/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6189 - accuracy: 0.6551
Epoch 124/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6193 - accuracy: 0.6574
Epoch 125/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6193 - accuracy: 0.6511
Epoch 126/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6201 - accuracy: 0.6511
Epoch 127/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6181 - accuracy: 0.6625
Epoch 128/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6188 - accuracy: 0.6591
Epoch 129/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6191 - accuracy: 0.6625
Epoch 130/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6178 - accuracy: 0.6540
Epoch 131/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6202 - accuracy: 0.6540
Epoc

55/55 [==============================] - 2s 6ms/step - loss: 0.6899 - accuracy: 0.5341
Epoch 2/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6857 - accuracy: 0.5545
Epoch 3/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6831 - accuracy: 0.5665
Epoch 4/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6756 - accuracy: 0.5682
Epoch 5/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6679 - accuracy: 0.5932
Epoch 6/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6630 - accuracy: 0.6017
Epoch 7/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6583 - accuracy: 0.5994
Epoch 8/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6557 - accuracy: 0.6034
Epoch 9/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6538 - accuracy: 0.6097
Epoch 10/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6546 - accuracy: 0.6119
Epoch 11/200
55/55 [=

55/55 [==============================] - 0s 6ms/step - loss: 0.6242 - accuracy: 0.6443
Epoch 165/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6237 - accuracy: 0.6506
Epoch 166/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6246 - accuracy: 0.6455
Epoch 167/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6241 - accuracy: 0.6477
Epoch 168/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6249 - accuracy: 0.6472
Epoch 169/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6249 - accuracy: 0.6386
Epoch 170/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6245 - accuracy: 0.6506
Epoch 171/200
55/55 [==============================] - 0s 6ms/step - loss: 0.6241 - accuracy: 0.6489
Epoch 172/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6245 - accuracy: 0.6489
Epoch 173/200
55/55 [==============================] - 0s 5ms/step - loss: 0.6245 - accuracy: 0.6472
Epoc

55/55 [==============================] - 0s 5ms/step - loss: 0.5409 - accuracy: 0.7307
Epoch 125/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5427 - accuracy: 0.7335
Epoch 126/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5416 - accuracy: 0.7364
Epoch 127/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5429 - accuracy: 0.7330
Epoch 128/200
55/55 [==============================] - 0s 6ms/step - loss: 0.5414 - accuracy: 0.7330
Epoch 129/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5443 - accuracy: 0.7330
Epoch 130/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5420 - accuracy: 0.7358
Epoch 131/200
55/55 [==============================] - 0s 5ms/step - loss: 0.5418 - accuracy: 0.7290
Epoch 132/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5425 - accuracy: 0.7341
Epoch 133/200
55/55 [==============================] - 0s 4ms/step - loss: 0.5432 - accuracy: 0.7301
Epoc

### 3X Data

In [108]:
# Using 3x data
trainlabel_list = [yva3, yar3, ydo3, yli3]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x_3.reshape(-1, 16,8),traain,test.reshape(-1, 16,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(2640, 16, 8, 1) (2640, 2)
Epoch 1/200
83/83 [==============================] - 2s 6ms/step - loss: 0.6882 - accuracy: 0.5489
Epoch 2/200
83/83 [==============================] - 0s 6ms/step - loss: 0.6652 - accuracy: 0.5826
Epoch 3/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6410 - accuracy: 0.5996
Epoch 4/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6316 - accuracy: 0.6083
Epoch 5/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6300 - accuracy: 0.6087
Epoch 6/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6244 - accuracy: 0.6201
Epoch 7/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6223 - accuracy: 0.6174
Epoch 8/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6226 - accuracy: 0.6216
Epoch 9/200
83/83 [==============================] - 0s 4ms/ste

83/83 [==============================] - 0s 5ms/step - loss: 0.6052 - accuracy: 0.6485
Epoch 163/200
83/83 [==============================] - 0s 6ms/step - loss: 0.6055 - accuracy: 0.6481
Epoch 164/200
83/83 [==============================] - 0s 6ms/step - loss: 0.6058 - accuracy: 0.6470
Epoch 165/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6054 - accuracy: 0.6455
Epoch 166/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6054 - accuracy: 0.6500
Epoch 167/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6054 - accuracy: 0.6485
Epoch 168/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6057 - accuracy: 0.6428
Epoch 169/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6057 - accuracy: 0.6492
Epoch 170/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6054 - accuracy: 0.6451
Epoch 171/200
83/83 [==============================] - 0s 6ms/step - loss: 0.6062 - accuracy: 0.6489
Epoc

83/83 [==============================] - 0s 5ms/step - loss: 0.6424 - accuracy: 0.6311
Epoch 123/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6433 - accuracy: 0.6337
Epoch 124/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6428 - accuracy: 0.6318
Epoch 125/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6430 - accuracy: 0.6330
Epoch 126/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6429 - accuracy: 0.6367
Epoch 127/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6423 - accuracy: 0.6322
Epoch 128/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6425 - accuracy: 0.6288
Epoch 129/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6420 - accuracy: 0.6273
Epoch 130/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6420 - accuracy: 0.6322
Epoch 131/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6418 - accuracy: 0.6318
Epoc

83/83 [==============================] - 2s 5ms/step - loss: 0.6879 - accuracy: 0.5545
Epoch 2/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6882 - accuracy: 0.5545
Epoch 3/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6814 - accuracy: 0.5735
Epoch 4/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6793 - accuracy: 0.5735
Epoch 5/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6740 - accuracy: 0.5788
Epoch 6/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6688 - accuracy: 0.5799
Epoch 7/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6670 - accuracy: 0.5799
Epoch 8/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6660 - accuracy: 0.5788
Epoch 9/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6623 - accuracy: 0.5962
Epoch 10/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6623 - accuracy: 0.6011
Epoch 11/200
83/83 [=

83/83 [==============================] - 0s 5ms/step - loss: 0.6468 - accuracy: 0.6265
Epoch 165/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6483 - accuracy: 0.6193
Epoch 166/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6465 - accuracy: 0.6227
Epoch 167/200
83/83 [==============================] - 0s 5ms/step - loss: 0.6504 - accuracy: 0.6140
Epoch 168/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6514 - accuracy: 0.6114
Epoch 169/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6486 - accuracy: 0.6227
Epoch 170/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6480 - accuracy: 0.6227
Epoch 171/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6470 - accuracy: 0.6223
Epoch 172/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6464 - accuracy: 0.6220
Epoch 173/200
83/83 [==============================] - 0s 4ms/step - loss: 0.6468 - accuracy: 0.6201
Epoc

83/83 [==============================] - 0s 5ms/step - loss: 0.5704 - accuracy: 0.7125
Epoch 125/200
83/83 [==============================] - 0s 5ms/step - loss: 0.5695 - accuracy: 0.7098
Epoch 126/200
83/83 [==============================] - 0s 5ms/step - loss: 0.5701 - accuracy: 0.7125
Epoch 127/200
83/83 [==============================] - 0s 6ms/step - loss: 0.5704 - accuracy: 0.7133
Epoch 128/200
83/83 [==============================] - 1s 6ms/step - loss: 0.5698 - accuracy: 0.7117
Epoch 129/200
83/83 [==============================] - 1s 6ms/step - loss: 0.5694 - accuracy: 0.7144
Epoch 130/200
83/83 [==============================] - 0s 5ms/step - loss: 0.5698 - accuracy: 0.7133
Epoch 131/200
83/83 [==============================] - 0s 5ms/step - loss: 0.5699 - accuracy: 0.7114
Epoch 132/200
83/83 [==============================] - 0s 5ms/step - loss: 0.5693 - accuracy: 0.7144
Epoch 133/200
83/83 [==============================] - 0s 5ms/step - loss: 0.5696 - accuracy: 0.7140
Epoc

### 4X Data

In [109]:
# Using 4x data
trainlabel_list = [yva4, yar4, ydo4, yli4]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 200 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x_4.reshape(-1, 16,8),traain,test.reshape(-1, 16,8),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(3520, 16, 8, 1) (3520, 2)
Epoch 1/200
110/110 [==============================] - 2s 5ms/step - loss: 0.6895 - accuracy: 0.5364
Epoch 2/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6706 - accuracy: 0.5739
Epoch 3/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6536 - accuracy: 0.5869
Epoch 4/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6455 - accuracy: 0.5946
Epoch 5/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6408 - accuracy: 0.5940
Epoch 6/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6401 - accuracy: 0.5969
Epoch 7/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6399 - accuracy: 0.5957
Epoch 8/200
110/110 [==============================] - 0s 5ms/step - loss: 0.6402 - accuracy: 0.5977
Epoch 9/200
110/110 [==========================

Epoch 80/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6298 - accuracy: 0.6151
Epoch 81/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6289 - accuracy: 0.6156
Epoch 82/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6298 - accuracy: 0.6187
Epoch 83/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6289 - accuracy: 0.6173
Epoch 84/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6312 - accuracy: 0.6148
Epoch 85/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6285 - accuracy: 0.6173
Epoch 86/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6280 - accuracy: 0.6170
Epoch 87/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6283 - accuracy: 0.6151
Epoch 88/200
110/110 [==============================] - 0s 4ms/step - loss: 0.6290 - accuracy: 0.6176
Epoch 89/200
110/110 [==============================] - 0s 5ms/step - loss: 0.6290

110/110 [==============================] - 1s 6ms/step - loss: 0.6574 - accuracy: 0.6074
Epoch 37/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6560 - accuracy: 0.6119
Epoch 38/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6561 - accuracy: 0.6170
Epoch 39/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6561 - accuracy: 0.6145
Epoch 40/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6556 - accuracy: 0.6162
Epoch 41/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6559 - accuracy: 0.6145
Epoch 42/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6566 - accuracy: 0.6116
Epoch 43/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6562 - accuracy: 0.6131
Epoch 44/200
110/110 [==============================] - 1s 7ms/step - loss: 0.6554 - accuracy: 0.6116
Epoch 45/200
110/110 [==============================] - 1s 6ms/step - loss: 0.6548 - accuracy: 

110/110 [==============================] - 1s 5ms/step - loss: 0.6517 - accuracy: 0.6179
Epoch 196/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6521 - accuracy: 0.6162
Epoch 197/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6517 - accuracy: 0.6170
Epoch 198/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6518 - accuracy: 0.6153
Epoch 199/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6516 - accuracy: 0.6153
Epoch 200/200
13/13 [==============================] - 0s 2ms/step
Accuracy 63.0
Loss 0.6706275343894958
######################################### Dominance #########################################
xtrain.shape,ytrain.shape
(3520, 16, 8, 1) (3520, 2)
Epoch 1/200
110/110 [==============================] - 2s 5ms/step - loss: 0.6895 - accuracy: 0.5480
Epoch 2/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6894 - accuracy: 0.5545
Epoch 3/200
110/110 [========================

110/110 [==============================] - 1s 5ms/step - loss: 0.6584 - accuracy: 0.6020
Epoch 153/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6594 - accuracy: 0.6006
Epoch 154/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6590 - accuracy: 0.6017
Epoch 155/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6588 - accuracy: 0.6003
Epoch 156/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6597 - accuracy: 0.6003
Epoch 157/200
110/110 [==============================] - 0s 4ms/step - loss: 0.6594 - accuracy: 0.5989
Epoch 158/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6594 - accuracy: 0.6003
Epoch 159/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6592 - accuracy: 0.6006
Epoch 160/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6599 - accuracy: 0.6014
Epoch 161/200
110/110 [==============================] - 1s 5ms/step - loss: 0.6594 - a

110/110 [==============================] - 1s 6ms/step - loss: 0.5849 - accuracy: 0.7054
Epoch 110/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5861 - accuracy: 0.7040
Epoch 111/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5859 - accuracy: 0.7043
Epoch 112/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5851 - accuracy: 0.7037
Epoch 113/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5849 - accuracy: 0.7045
Epoch 114/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5847 - accuracy: 0.7034
Epoch 115/200
110/110 [==============================] - 1s 6ms/step - loss: 0.5851 - accuracy: 0.7048
Epoch 116/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5865 - accuracy: 0.7051
Epoch 117/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5846 - accuracy: 0.7043
Epoch 118/200
110/110 [==============================] - 1s 5ms/step - loss: 0.5842 - a